In [1]:
image_dir = '/home/dell/Desktop/My_learning/DL_tutorial/keras_functional_model/archive/UTKFace'

In [2]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

2022-10-23 11:20:30.394063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 11:20:30.766703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-23 11:20:30.766728: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-23 11:20:30.804442: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 11:20:31.619484: W tensorflow/stream_executor/platform/de

In [3]:
data = []
for image in os.listdir(image_dir):
    age = int(image.split('_')[0])
    gender = int(image.split('_')[1])
    data.append((age,gender,image))

In [4]:
df = pd.DataFrame(data,columns=['age','gender','image_name'])

In [5]:
df

,age,gender,image_name
0,26,0,26_0_2_20170116184500032.jpg.chip.jpg
1,27,0,27_0_3_20170119193359595.jpg.chip.jpg
2,37,0,37_0_1_20170116205022380.jpg.chip.jpg
3,23,1,23_1_2_20170116172830806.jpg.chip.jpg
4,1,0,1_0_3_20161219230232072.jpg.chip.jpg
...,...,...,...
23703,26,0,26_0_3_20170119175254874.jpg.chip.jpg
23704,85,1,85_1_0_20170120230302610.jpg.chip.jpg
23705,28,1,28_1_2_20170109002652355.jpg.chip.jpg
23706,32,0,32_0_2_20170116191450850.jpg.chip.jpg


In [6]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=image_dir,
                                                    x_col='image_name',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=image_dir,
                                                    x_col='image_name',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [11]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

2022-10-23 11:21:31.041402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-23 11:21:31.041689: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-23 11:21:31.041721: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kunal-latitude-3510): /proc/driver/nvidia/version does not exist
2022-10-23 11:21:31.042025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [13]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [14]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [15]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
 18/625 [..............................] - ETA: 18:58 - loss: 532.3419 - age_loss: 18.1567 - gender_loss: 5.1938 - age_mae: 18.1567 - gender_accuracy: 0.4983

KeyboardInterrupt: 